# Multimodal Early Fusion

## Requirements

Multimodal: Given a query, these retrieval systems selects the tracks that are most similar to the query track, based on more
than one complementary feature (i.e., of different modalities). For this purpose, you will use fusion strategies based on
* Late fusion
*  Early fusion
You will hence implement two different multimodal retrieval systems

In [1]:
import pandas as pd
import torch
import os, sys
from tqdm import tqdm

target_folder = 'MMSR25-26-Group-E'
current_path = os.getcwd()

while os.path.basename(current_path) != target_folder:
    if os.path.basename(current_path) == 'RetrievalAlgorithm':
        if os.path.join(current_path) not in sys.path:
            sys.path.append(os.path.join(current_path))
    parent = os.path.dirname(current_path)
    os.chdir(parent)
    current_path = parent

from RetrievalAlgorithm.src.utils.data_loading import load_all_tsv_files_from_path
from RetrievalAlgorithm.src.score_calculation_modules.cosine_similarity_module import CosineSimilarityModule
from RetrievalAlgorithm.src.multimodal_early_fusion_calculations import calculate_multimodal_similarity

## Module initialization

In [2]:
unimodal_module_path = 'RetrievalAlgorithm/modules/cosine_similarity_module.pt'

if os.path.exists(unimodal_module_path):
    unimodal_module = torch.jit.load(unimodal_module_path)
else:
    unimodal_module = CosineSimilarityModule()
    unimodal_module = torch.jit.script(unimodal_module)
    unimodal_module.save(unimodal_module_path)

## Data Loading

In [3]:
dataset_files_dict = load_all_tsv_files_from_path(path_to_dataset='Dataset')

Loading .tsv files: 100%|██████████| 9/9 [00:00<00:00, 997.54it/s]


## Calculating Multimodal Similarity Scores

### Lyrics + Audio

In [4]:
datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                 dataset_files_dict['id_mfcc_bow_mmsr.tsv']]

lyrics_audio_similarity_scores_df = calculate_multimodal_similarity(
    datasets_df=datasets_list,
    calculation_module=unimodal_module,
    feature_name='Lyrics_Audio',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [02:17<00:00, 61.03it/s]


### Lyrics + Videoclips

In [5]:
datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                 dataset_files_dict['id_vgg19_mmsr.tsv']]

lyrics_videoclips_similarity_scores_df = calculate_multimodal_similarity(
    datasets_df=datasets_list,
    calculation_module=unimodal_module,
    feature_name='Lyrics_Videoclips',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [09:40<00:00, 14.46it/s]


### Audio + Videoclips

In [6]:
datasets_list = [dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
                 dataset_files_dict['id_vgg19_mmsr.tsv']]

audio_videoclips_similarity_scores_df = calculate_multimodal_similarity(
    datasets_df=datasets_list,
    calculation_module=unimodal_module,
    feature_name='Audio_Videoclips',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [09:15<00:00, 15.11it/s]


### Lyrics + Audio + Videoclips

In [7]:
datasets_list = [dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
                 dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
                 dataset_files_dict['id_vgg19_mmsr.tsv']]

lyrics_audio_videoclips_similarity_scores_df = calculate_multimodal_similarity(
    datasets_df=datasets_list,
    calculation_module=unimodal_module,
    feature_name='Lyrics_Audio_Videoclips',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [09:57<00:00, 14.06it/s]


## Merge and Save results

In [8]:
sim_scores_df_list = [
    ('lyrics_audio', lyrics_audio_similarity_scores_df),
    ('lyrics_videoclips', lyrics_videoclips_similarity_scores_df),
    ('audio_videoclips', audio_videoclips_similarity_scores_df),
    ('lyrics_audio_videoclips', audio_videoclips_similarity_scores_df),
]

target_dir = 'RetrievalAlgorithm/results/multimodal/early_fusion/raw'
os.makedirs(target_dir, exist_ok=True)

for feature_name, sim_scores_df in tqdm(sim_scores_df_list, desc='Saving similarity scores'):
    output_path = os.path.join(target_dir, f'early_fusion_{feature_name}_similarity_scores.parquet')
    sim_scores_df.to_parquet(output_path, index=False)

Saving similarity scores: 100%|██████████| 4/4 [00:10<00:00,  2.72s/it]
